

## **Cleaning Data with Pyspark**

Welcome to this hands-on training where we will investigate cleaning a dataset using Python and Apache Spark! During this training, we will cover:

* Efficiently loading data into a Spark DataFrame
* Handling errant rows / columns from the dataset, including comments, missing data, combined or misinterpreted columns, etc.
* Using Python UDFs to run advanced transformations on data


## **The Dataset**

The dataset used in this webinar is a set of CSV files named `netflix_titles_raw*.csv`. These contain information related to the movies and television shows available on Netflix. These are the *dirty* versions of the dataset - we will cover the individual problems as we work through the notebook.

Given that this is a data cleaning webinar, let's look at our intended result.  The dataset will contain the follwing information:

- `show_id`: A unique integer identifier for the show
- `type`: The type of content, `Movie` or `TV Show`
- `title`: The title of the content
- `director`: The director (or directors)
- `cast`: The cast
- `country`: Country (or countries) where the content is available
- `date_added`: Date added to Netflix
- `release_year`: Year of content release
- `rating`: Content rating
- `duration`: The duration
- `listed_in`: The genres the content is listed in
- `description`: A description of the content



## **Setting up a PySpark session**

Before we can start processing our data, we need to configure a Pyspark session for Google Colab. Note that this is specific for using Spark and Python in Colab and likely is not required for other environments. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys

# Import spark session
import pyspark
from pyspark.sql.types import *
from pyspark.sql import functions as sf
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import year
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DoubleType
from pyspark.sql.types import DateType
from pyspark.sql.functions import * #sum, mean,col, udf
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.linalg import Vectors
import datetime

spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "16G") \
    .appName('pyspark-app-1') \
    .getOrCreate()

sqlContext = SQLContext(spark)

## **Loading our initial DataFrame**

Let's take a look at what Spark does with our data and see if it can properly parse the output. To do this, we'll first load the content into a DataFrame using the `spark.read.csv()` method. 

We'll pass in three arguments:

 - The path to the file(s)
 - An entry for `header=False`. Our files do not have a header row, so we must specify this or risk a data row being interpreted as a header.
 - The last argument we add is the `sep` option, which specifies the field separator. Often in CSV files this is a comma (`,`), but in our files it's a `\t` or tab character.

In our command, we'll be using the wildcard character, `*`, again to access all the files matching `/tmp/netflix_titles_dirty*.csv.gz` (ie, all the files we've downloaded thus far). Spark will handle associating all the files to the same dataframe. 

Depending on how familiar you are with Spark, you should note that this line does not actually read the contents of the files yet. This command is *lazy*, which means it's not executed until we request Spark to execute some type of analysis. This is a crucial point to understand how data processing behaves within Spark. 

In [3]:
# Read csv file wiith spark.read_csv()

titles_df = spark.read.csv('C:/Users/dennis.ee/Desktop/Directory/Demo/PySpark Data Cleaning\data/netflix_titles_dirty*.csv', header=False, sep='\t')

## **Initial analysis**

Let's look at the first 150 rows using the `.show()` method on the DataFrame. We'll pass in:

- The number of rows to display (`150`)
- Set the `truncate` option to `False` so we can see all the DataFrame columns and content.

*Note: `.show()` is a Spark action, meaning that any previous lazy commands will now be executed prior to the actual action. Spark can also optimize these commands for best performance as needed.*



In [4]:
# Show first 150 rows

titles_df.show(150, truncate=False)

+------------------------------------------------------------------------------+-------+-----------------------------------------------+-------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------+-------------+----+-----+--------+------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------


**Observation 1**:

If we start by looking at the first column of data, we see that most of the entries are numeric IDs. If we scroll through the data we do see at least one random text entry. Let's make a note of this and browse through the rest of the data, looking for anything that might be out of the ordinary.

**Problem 1**: First column contains a mix of numeric and string data.

**Observation 2**:
We can also use the `.printSchema()` method to print the inferred schema associated with the data. Notice that we have 12 columns (which is expected based on our format information) but there are no column names, incorrect datatypes, and each field is nullable. *Note: `.printSchema()` is also a Spark action.*

**Problem 2**: Column names are generic.

**Problem 3**: All columns are typed as strings, but appear to contain various datatypes (also reference **problem 1**)

In [5]:
# Print schema

titles_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)



## **Bypassing the CSV interpeter**

Our first few data rows look ok, but we can see that we have a few random rows even in our small sample. We know the first column should be an integer value but it looks like there are some values that do not meet this requirement. 

Let's run a `.count()` method on our dataframe to determine how many rows are present in the dataset, regardless of whether they're correct.


In [6]:
# Count the number of rows

titles_df.count()

6238

 Let's run a quick select statement on the DataFrame to determine the makeup of the content. We'll use some methods from the `pyspark.sql.functions` module to help us determine this. We'll alias this module as `F` for simplicity.

We're going to use the `F.col` method to find the column named `_c0` in our dataframe. We'll then chain the function `.cast("int")` to attempt to change each entry from a string value to an integer one. We then use the `.isNotNull()` function to find only entries that are not null. This is passed to the the `.filter()` method on the dataframe to return only rows that meet this requirement. Finally we run the `.count()` method on the resulting dataframe to get the count of rows where the first column is an integer value.

An example to filter a Spark dataframe named `df_1` with a column named `_c0` would be:

```
from pyspark.sql import functions as F

df_1.filter(F.col("_c0").cast("int").isNotNull()).show()
```


In [6]:
# Import the Pyspark SQL helper functions
from pyspark.sql import functions as F

# Determine how many rows have a column that converts properly to an integer value
titles_df.filter(F.col("_c0").cast("int").isNotNull()).count()


6173

### **Spark is different**

Depending on your background, you may be very confused as to what's going on right now. If you're used to pandas, you should know that Spark behaves similarly, but there are some significant differences.

#### **Spark dataframes are immutable**
- This means that once a dataframe is created, you cannot change the contents of the dataframe, you can only create a new one.
- We won't cover why during this class, beyond mentioning that it makes a distributed system (such as Spark) much more manageable.
- While you cannot change a dataframe, you can create a new one.
- Dataframes in Spark are defined with various types of *transformations*. These are the *lazy* commands we mentioned earlier. You can think of them as a recipe, or set of commands that will be run at a given time. 
- Spark *actions* (such as `.count()`, `.show()`, `.write()`, among others) instantiate the dataframe, meaning that the data is processed and available within the dataframe.

The important thing to note is that these `.filter()` commands are not changing our underlying dataframe - it's creating a new dataframe with the results of the `.filter()` operation and giving us the `.count()` of that. This is all done behind the scenes for you, but it's important to understand what Spark is doing underneath.

### **Back to analysis**

Now that we've determined there is a difference between the number of correct entries vs all entries (ie, take the full count and subtract the filtered dataframe), let's look at the rows that aren't coverting properly.

We'll do the same as before, but this time we're going to use the `.isNull()` method to obtain only the rows that can't be cast to integers. We'll use the `.show()` method to see the content.

*Note, this is only possible because Spark maintains an immutable dataframe. In our previous step, we created a new dataframe from our `.filter()` command, ran an action, and then Spark threw away the dataframe as we didn't assign it to a variable. We're going to now do the same type of operation on the original `titles_df` dataframe.*

In [7]:
# Look at rows that don't convert properly
titles_df.filter(F.col("_c0").cast("int").isNull()).show(truncate=False)

+-----------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+--------------+-----+--------+----------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------+----+----+
|_c0                                                                                        

**Observation 3**

Looking at our content, we notice we have several types of problems:

*Data problems*:
- **Problem 4**: Comment rows - These begin with a `#` character in the first column, and all other columns are null
- **Problem 5**: Missing first column - We have few rows that reference `TV Show` or `Movie`, which should be the 2nd column.
- **Problem 6**: Odd columns - There are a few rows included where the columns seem out of sync (ie, a content type in the ID field, dates in the wrong column, etc).

We could fairly easily remove rows that match this pattern, but we're not entirely sure what to expect here. This is a common issue when trying to parse a large amount of data, be it in native Python, in Spark, or even with command-line tools. 

What we need to do is bypass most of the CSV parser's intelligence, but still load the content into a DataFrame. One way to do this is to modify an option on the CSV loader.

# **CSV loading**

Our initial import relies on the defaults for the CSV import mechanism. This typically assumes an actual comma-separated value file using `,` between fields and a normal row level terminator (ie, `\r\n`, `\r`, `\n`). While this often works well, it doesn't always handle ever data cleaning process you'd like, especially if you want to save the errant data for later examination.

One way we can trick our CSV load is to specify a custom separator that we know does not exist within our dataset. As we used above, the option to do this is called `sep` and takes a single character to be used as the column separator. The separator cannot be an empty string so depending on your data, you may need to determine a character that is not used. For our purposes, let's use a curly brace, `{`, which is most likely not present in our data.

In [8]:
# Load the files into a DataFrame with a single column

titles_single_df = spark.read.csv('C:/Users/dennis.ee/Desktop/Directory/Demo/PySpark Data Cleaning\data/netflix_titles_dirty*.csv', sep='{')

In [9]:
# Count rows

titles_single_df.count()

6238

In [10]:
# Show header

titles_single_df.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|_c0                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
+-----------------------------------------------------------------------------------------------------------------

In [11]:
# Print schema

titles_single_df.printSchema()

root
 |-- _c0: string (nullable = true)



# **Q&A?**

# **Cleaning up our data**

We know from some earlier analysis that we have comment rows in place (ie, rows that begin with a `#`). While Spark provides a DataFrame option to handle this automatically, let's consider what it would take to remove comment rows.

We need to:

- Determine if the column / line starts with a `#`
- If so, filter these out to a new DataFrame

There are many ways to accomplish this in Spark, but let's use a conceptually straight-forward option, `.startsWith()`.

In [12]:
# Filter DataFrame and show rows that starts with #

titles_single_new_df = titles_single_df.filter(F.col('_c0').startswith('#'))
titles_single_new_df.show()

+--------------------+
|                 _c0|
+--------------------+
|# (87.8 percent B...|
|# On exteriority,...|
|# Roosevelt Unive...|
|# An "or". a conv...|
|# 1611–1613 Kalma...|
|# A legally scien...|
|# Reparenting, ma...|
| # USA South is done|
|# The wave for is...|
|# Which was, Chan...|
|# Five 40 dangero...|
|# Heisei Hyakkei ...|
|# (or will advanc...|
|# Procedures. Kow...|
|# 3 million. part...|
|# And 1985, by bo...|
|# Their decision,...|
|# 82 (4): birds h...|
|# Viva was surfac...|
|# Of Monte-Cristo...|
+--------------------+
only showing top 20 rows



In [13]:
# Count number of rows

titles_single_new_df.count()

47

We've determined that we have 47 rows that begin with a comment character. We can now easily filter these from our DataFrame as we do below. This resolves **Problem 4**.

*Note: We're doing things in a more difficult fashion than is absolutely necessary to illustrate options. The Spark CSV reader has an option for a `comment` property, which actually defaults to skipping all rows starting with a `#` character. That said, it only supports a single character - consider if you were looking for multi-character options (ie, a // from C-style syntax). This feature is also only available in newer versions of Spark, where our method works in any of the 2.x Spark releases.*

In [14]:
# Filter out comments

titles_single_df = titles_single_df.filter(~ F.col('_c0').startswith('#'))

In [15]:
titles_single_df.count()

6191

# **Checking column counts**

Our next step for cleaning this dataset in Pyspark involves determining how many columns we should have in our dataset and take care of any outliers. We know from our earlier examination that the dataset should have 12 usable columns per row.

First, let's determine how many columns are present in the data and add that as a column. We'll do this with a combination of:

- `F.split()`: This acts similar to the Python `split()` method, splitting the contents of a dataframe column on a specified character into a Spark ArrayType column (ie, Spark's version of a list variable). 
- `F.size()`: Returns the size (length) of a Spark ArrayType column.
- `.withColumn()`: Creates a new dataframe with a given column.

In [16]:
# Add a column representing the total number of fields / columns

titles_single_df = titles_single_df.withColumn('fieldCount', F.size(F.split(titles_single_df['_c0'], '\t')))

In [17]:
# Show rows with a fieldcount > 12 (Note, select statement here isn't necessarily required - used to reorder the columns for easier viewing)
titles_single_df.select('fieldcount', '_c0').where('fieldCount > 12').show(truncate=False)

+----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|fieldcount|_c0                                                                                                        

In [18]:
# Check for any rows with fewer than 12 columns

titles_single_df.select('fieldcount', '_c0').where('fieldcount < 12').show(truncate=False)

+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|fieldcount|_c0                                                                                                                                                                                                                                                                                                                                                                                        

**Observation 4**: We notice that we have many rows with fewer or greater than 12 columns of data. 

**Problem 7**: Column counts do not match our expected schema.

In [18]:
# Save these to a separate dataframe for later analysis

titles_badrows_df = titles_single_df.where('fieldcount != 12')

In [19]:
# Determine total number of "bad" rows
titles_badrows_df.count()

78

We can now resolve **problems 5, 6, and 7** by accessing only the rows that have 12 columns present.

In [20]:
# Set the dataframe without the bad rows
titles_single_df = titles_single_df.where('fieldcount == 12')

In [21]:
# How many current rows in dataframe?

titles_single_df.count()

6113

# **Q&A**

# **More cleaning / prep**

Now that we've removed rows that don't fit our basic formatting, let's continue on with making our dataframe more useful.

First, let's create a new column that is a list (actually a Spark ArrayType column) containing all "columns" using the `pyspark.sql.functions.split` method. We'll call this `splitcolumn`.

In [22]:
# Create a list of split strings as a new column named splitcolumn
titles_cleaned_df = titles_single_df.select(F.split('_c0', '\t').alias('splitcolumn'))


In [23]:
# View the contents
titles_cleaned_df.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|splitcolumn                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
+---------------------------------------------------------------------------------------

# **Creating typed columns**

There are several ways to do this operation depending on your needs, but for this dataset we'll explicitly convert the strings in the ArrayType column (ie, a Spark list column) to actual dataframe columns. The `.getItem()` method returns the value at the specified index of the listcolumn (ie, of `splitcolumn`). 

Let's consider if we wanted to create a full dataframe from the following example dataframe (`df_1`):

splitcolumn |
---|
[1,USA,North America]
[2,France,Europe]
[3,China,Asia]

```
df_1 = df_1.withColumn('country_id', df_1.splitcolumn.getItem(0).cast(IntegerType())
df_1 = df_1.withColumn('country_name', df_1.splitcolumn.getItem(1))
df_1 = df_1.withColumn('continent', df_1.splitcolumn.getItem(2))
```

This would give you a resulting dataframe of:

splitcolumn | country_id | country_name | continent
---|---|---|---
[1,USA,North America]|1|USA|North America
[2,France,Europe]|2|France|Europe
[3,China,Asia]|3|China|Asia

The `splitcolumn` is currently still present - we'll take care of that later on.

Note that for `show_id` and `release_year`, we'll also use `.cast()` to specify them as integers rather than just strings. We also need to import the `IntegerType` from the `pyspark.sql.types` module to properly cast our data to an integer column in Spark.



In [24]:
from pyspark.sql.types import IntegerType

# Create columns with specific data types using .getItem()

titles_cleaned_df = titles_cleaned_df.withColumn('show_id', titles_cleaned_df.splitcolumn.getItem(0).cast(IntegerType()))
titles_cleaned_df = titles_cleaned_df.withColumn('type', titles_cleaned_df.splitcolumn.getItem(1))
titles_cleaned_df = titles_cleaned_df.withColumn('title', titles_cleaned_df.splitcolumn.getItem(2))
titles_cleaned_df = titles_cleaned_df.withColumn('director', titles_cleaned_df.splitcolumn.getItem(3))
titles_cleaned_df = titles_cleaned_df.withColumn('cast', titles_cleaned_df.splitcolumn.getItem(4))
titles_cleaned_df = titles_cleaned_df.withColumn('country', titles_cleaned_df.splitcolumn.getItem(5))
titles_cleaned_df = titles_cleaned_df.withColumn('date_added', titles_cleaned_df.splitcolumn.getItem(6))
titles_cleaned_df = titles_cleaned_df.withColumn('release_year', titles_cleaned_df.splitcolumn.getItem(7).cast(IntegerType()))
titles_cleaned_df = titles_cleaned_df.withColumn('rating', titles_cleaned_df.splitcolumn.getItem(8))
titles_cleaned_df = titles_cleaned_df.withColumn('duration', titles_cleaned_df.splitcolumn.getItem(9))
titles_cleaned_df = titles_cleaned_df.withColumn('listed_in', titles_cleaned_df.splitcolumn.getItem(10))
titles_cleaned_df = titles_cleaned_df.withColumn('description', titles_cleaned_df.splitcolumn.getItem(11))

Let's now drop our columns that aren't needed anymore. These are `_c0` (the original single line string), `fieldcount`, and `splitcolumn`. You can drop these as a single column per entry, or a comma-separated set of column names.

In [25]:
# Drop original _c0 column

titles_cleaned_df = titles_cleaned_df.drop('_c0', 'fieldcount', 'splitcolumn')

Let's verify our content, check the row count, and then look at our schema.

In [26]:
# Showcase new DataFrame

titles_cleaned_df.show()

+--------+-------+--------------------+----------------+--------------------+--------------------+-------------+------------+------+--------+--------------------+--------------------+
| show_id|   type|               title|        director|                cast|             country|   date_added|release_year|rating|duration|           listed_in|         description|
+--------+-------+--------------------+----------------+--------------------+--------------------+-------------+------------+------+--------+--------------------+--------------------+
|80044126|  Movie| D.L. Hughley: Clear|     Jay Chapman|        D.L. Hughley|       United States|July 13, 2017|        2014| TV-MA|  59 min|     Stand-Up Comedy|In this 2014 stan...|
|80148179|  Movie|My Scientology Movie|      John Dower|       Louis Theroux|      United Kingdom|July 13, 2017|        2015| TV-MA|  99 min|       Documentaries|After speaking wi...|
|70301023|  Movie|Tom Segura: Compl...|     Jay Chapman|          Tom Segura|   

In [27]:
# Count rows

titles_cleaned_df.count()

6113

In [28]:
# Print the schema

titles_cleaned_df.printSchema()

root
 |-- show_id: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



# **Even more cleanup**

With our last set of steps, we've successfully fixed our remaining **problems (1, 2, and 3)**. Now that we have a generally clean dataset, let's look for further issues.

If we look at the distinct values available in the show `type` column, we see an issue. This is a categorical data column - collapsing the values in this column is a typical data cleaning issue.

In [29]:
# Check out unique items in type column

titles_cleaned_df.select('type').distinct().show()

+-------+
|   type|
+-------+
|TV Show|
|  Movie|
|       |
+-------+



In [30]:
# Isolate rows where type is ""

titles_cleaned_df.where('type == ""').show()

+--------+----+--------------------+--------------+--------------------+----------------+----------------+------------+------+--------+--------------------+--------------------+
| show_id|type|               title|      director|                cast|         country|      date_added|release_year|rating|duration|           listed_in|         description|
+--------+----+--------------------+--------------+--------------------+----------------+----------------+------------+------+--------+--------------------+--------------------+
|80156212|    |           Wakefield| Robin Swicord|Bryan Cranston, J...|   United States|   March 2, 2019|        2016|     R| 109 min|              Dramas|An unhappy father...|
|80219180|    |Darr Sabko Lagta Hai|              |        Bipasha Basu|           India|   March 1, 2018|        2015| TV-MA|1 Season|International TV ...|In this chilling ...|
|80032097|    |        Romeo Ranjha|Navaniat Singh|Jazzy B, Garry Sa...|   India, Canada|November 1, 2017|    

You'll notice that we have 5 rows where the type is not specified when one should be. 

**Problem 8**: Invalid entry in a column - The type column should contain only `TV Show` or `Movie`, but it also contains an empty string value.


 We have a couple of options:

- Drop the rows
- Infer what the `type` is based on other content available in the dataset

You could remove the problem rows using something like:

```
titles_cleaned_df = titles_cleaned_df.where('type == "TV Show" or type == "Movie"')
```

That feels a bit like cheating though - let's consider how else we could determine this largely automatically.

If you look at the `duration` column, you'll notice that there are different meanings behind the entries. We have durations that contain the word *min* (minutes) or the word `Season` for seasons of the show. We can try to use these to properly set the `type` value for these rows.

This problem is a bit tricky though as Spark does not have the concept of updating data within a column without jumping through several hoops. We can however work through this issue using a User Defined Function, or UDF.

## **UDF**

If you haven't worked with them before, a UDF is a Python function that gets applied to every row of a DataFrame. They are extremely flexible and can help us work through issues such as our current one.

A UDF in Pyspark requires three things:

1. **A Python function or callable**: This is the function that you want called when the UDF is run by Spark.
2. **A UDF variable**: Defined by the `udf()` function, with the Python callable defined in #1, and the Spark return type.
3. **A Spark transformation**: The UDF must be defined via a transformation (ie `.withColumn()`) to be applied to the dataframe.

Consider the following dataframe `df_1`, containing a two fields, `a` and `b`

a|b
---|---
1|2
2|3
3|4

For illustration, let's say we want to use a UDF to define a new column, which is simply the value `a*b`, unless the value of `a` is `3`. If it is, then we want the value to be `0`.

Let's define our function - taking two arguments, `a`, and `b`. 

```
def multiply(a, b):
  if a == 3:
    return 0
  else:
    return a*b
```

Now, we need to define our UDF variable. We need to import `udf` from `pyspark.sql.functions`, and as we're returning an integer value, we need to import `IntegerType` from `pyspark.sql.types`. 

```
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

udfMultiply = udf(multiply, IntegerType())
```

Note that the `udf` function only takes the name of the callable, not any arguments. Those are defined with our last step - defining a new column. 

```
df_1 = df_1.withColumn('output', udfMultiply(F.col('a'), F.col('b')))
```

Note that in this instance we're using the `F.col` method to refer to the column. You could use any of the other valid Spark methods to specify a dataframe column (ie, `df_1.a`, `df_1['b']`, etc).

Assuming we run a Spark action, such as `.show()`, we would get a dataframe with the following contents:

a|b|output
---|---|---
1|2|2
2|3|6
3|4|0

*Note this is a trivial example for illustration purposes - the same behavior would be much better implemented using assorted functions in the Spark libraries.*


First, let's define our Python function that takes two arguments, a showtype (ie, *Movie*, *TV Show*, or other) and the showduration. Note that these are strings in this case. We'll check if the showtype is already a Movie or TV Show - if so, just return that value. Otherwise, we'll check if the showduration ends with *min*, indicating a Movie. If not, we'll specify it as a TV Show.


In [31]:
# Define the UDF callable

def deriveType(showtype, showduration):
  if showtype == 'Movie' or showtype == 'TV Show':
    return showtype
  else:
    if showduration.endswith('min'):
      return "Movie"
    else:
      return "TV Show"

# **Define the UDF for Spark**

Now we need to configure the UDF for Spark to access it accordingly.

In [32]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

udfDeriveType = udf(deriveType, StringType())

In [33]:
# Create a new derived column, passing in the appropriate values

titles_cleaned_df = titles_cleaned_df.withColumn('derivedType', udfDeriveType(F.col('type'), F.col('duration')))

In [34]:
# Show the rows where type is an empty string again, examining the derivedType
titles_cleaned_df.where('type == ""').show()

+--------+----+--------------------+--------------+--------------------+----------------+----------------+------------+------+--------+--------------------+--------------------+-----------+
| show_id|type|               title|      director|                cast|         country|      date_added|release_year|rating|duration|           listed_in|         description|derivedType|
+--------+----+--------------------+--------------+--------------------+----------------+----------------+------------+------+--------+--------------------+--------------------+-----------+
|80156212|    |           Wakefield| Robin Swicord|Bryan Cranston, J...|   United States|   March 2, 2019|        2016|     R| 109 min|              Dramas|An unhappy father...|      Movie|
|80219180|    |Darr Sabko Lagta Hai|              |        Bipasha Basu|           India|   March 1, 2018|        2015| TV-MA|1 Season|International TV ...|In this chilling ...|    TV Show|
|80032097|    |        Romeo Ranjha|Navaniat Singh

In [36]:
# Drop the original type column and rename derviedType to type
titles_cleaned_df = titles_cleaned_df.drop('type').withColumnRenamed('derivedType', 'type')

In [37]:
# Verify we only have two types available
titles_cleaned_df.select('type').distinct().show()

AnalysisException: cannot resolve '`type`' given input columns: [cast, country, date_added, description, director, duration, listed_in, rating, release_year, show_id, title];;
'Project ['type]
+- Project [show_id#308, title#315, director#320, cast#326, country#333, date_added#341, release_year#350, rating#360, duration#371, listed_in#383, description#396]
   +- Project [show_id#308, title#315, director#320, cast#326, country#333, date_added#341, release_year#350, rating#360, duration#371, listed_in#383, description#396, derivedType#548 AS type#629]
      +- Project [show_id#308, title#315, director#320, cast#326, country#333, date_added#341, release_year#350, rating#360, duration#371, listed_in#383, description#396, derivedType#548]
         +- Project [show_id#308, type#311, title#315, director#320, cast#326, country#333, date_added#341, release_year#350, rating#360, duration#371, listed_in#383, description#396, deriveType(type#311, duration#371) AS derivedType#548]
            +- Pro

In [38]:
# Verify our row count is the same
titles_cleaned_df.count()

6113

We've now successfully resolved **problem 8**.

# **Saving data for analysis / further processing**

The last step of our data cleaning is to save the cleaned dataframe out to a file type. If you plan to do any further analysis or processing using Spark, it's highly recommended you use Parquet. Other options are available per your needs, but Spark is optimized to take advantage of Parquet.

There are two options we use for the `.write.parquet()` method:

- The path of where to write the file
- An optional `mode` parameter, which we've set to `overwrite`. This allows Spark to write data to an existing location, solving some potential issues in a notebook environment.

In [39]:
titles_cleaned_df.show()

+--------+--------------------+----------------+--------------------+--------------------+-------------+------------+------+--------+--------------------+--------------------+-------+
| show_id|               title|        director|                cast|             country|   date_added|release_year|rating|duration|           listed_in|         description|   type|
+--------+--------------------+----------------+--------------------+--------------------+-------------+------------+------+--------+--------------------+--------------------+-------+
|80044126| D.L. Hughley: Clear|     Jay Chapman|        D.L. Hughley|       United States|July 13, 2017|        2014| TV-MA|  59 min|     Stand-Up Comedy|In this 2014 stan...|  Movie|
|80148179|My Scientology Movie|      John Dower|       Louis Theroux|      United Kingdom|July 13, 2017|        2015| TV-MA|  99 min|       Documentaries|After speaking wi...|  Movie|
|70301023|Tom Segura: Compl...|     Jay Chapman|          Tom Segura|       Unit

In [40]:
titles_cleaned_df.printSchema()

root
 |-- show_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)
 |-- type: string (nullable = true)



In [41]:
titles_cleaned_df = titles_cleaned_df.na.fill(value=0)

In [42]:
titles_cleaned_df.where(col("release_year").isNull()).show()

+-------+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+----+
|show_id|title|director|cast|country|date_added|release_year|rating|duration|listed_in|description|type|
+-------+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+----+
+-------+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+----+



In [43]:
titles_cleaned_df.where(col("type").isNull()).show()

+-------+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+----+
|show_id|title|director|cast|country|date_added|release_year|rating|duration|listed_in|description|type|
+-------+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+----+
+-------+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+----+



In [45]:
titles_cleaned_df.toPandas().to_csv('netflix_titles_cleaned_newv2.csv',index=False)